## Extending ProStruct with Python

In [1]:
import prostruct
import numpy as np
import math

# pure python functions are faster than numpy for 
# small vectors/non vectorised functions 
def dot3(a, b):
    return sum([(a_i*b_i) for a_i, b_i in zip(a, b)])

def cross3(a, b):
    return [a[1]*b[2] - a[2]*b[1], a[2]*b[0]-a[0]*b[2], a[0]*b[1]-a[1]*b[0]]

def dihedral(a1, a2, a3, a4):
    d1 = a1-a2
    d2 = a2-a3
    d3 = a3-a4
    b1 = (d1) / np.linalg.norm(d1)
    b2 = (d2) / np.linalg.norm(d2)
    b3 = (d3) / np.linalg.norm(d3)
    n1 = cross3(b1, b2)
    n2 = cross3(b2, b3)
    return math.degrees(math.atan2(dot3(cross3(n1, b2), n2), dot3(n1, n2)))

class MyStruct(prostruct.CustomPDB): 
    def custom_kernel(self, a, b):
        a = a.get_backbone_atoms()
        b = b.get_backbone_atoms()
        return dihedral(a[2], b[0], b[1], b[2])

In [2]:
pdb = MyStruct("../../../test.pdb")

In [3]:
pdb

<prostruct.PDB float precision, with 1867 atoms, 236 residues at 0x55619f2d22a0>

In [4]:
# with python implementation
%timeit pdb.run_custom_kernel()

8.94 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
# much faster (almost 3 orders of magnitude faster)
%timeit pdb.calculate_phi()

12.4 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [6]:
# same result though
pdb.run_custom_kernel()

array([ -69.337006, -133.41187 , -120.3177  , -102.3891  ,  -94.71912 ,
       -138.65765 ,  -66.235054,  -76.0483  , -139.28873 , -136.68294 ,
        -76.06559 , -143.39508 ,  -66.428116,  -62.458706,   82.80521 ,
        -64.59577 ,  -82.87559 ,  -98.586426, -134.31032 , -122.848854,
       -101.42037 , -157.77371 , -110.94793 ,  -91.86946 ,  -71.31498 ,
       -154.61761 ,  -57.16444 , -100.23258 ,  -67.524   ,  -77.13684 ,
        -55.50057 ,  -99.62578 ,   86.937645, -103.419136, -119.52794 ,
        -71.52652 , -119.19695 , -130.72379 , -146.96056 , -140.68657 ,
       -125.35315 , -110.42541 ,  -76.84396 ,  -49.87192 ,   68.4221  ,
        -95.02083 ,  -57.79741 ,  -63.16061 , -134.48965 ,  -59.8783  ,
       -119.90971 , -144.67488 , -159.37997 ,   35.841885,   82.76004 ,
       -137.41612 , -117.15545 , -102.0791  ,  -62.144516,  -60.305027,
         91.5626  ,  -78.7855  ,  -65.14505 ,  -62.12854 ,  -57.483463,
        -76.94548 , -142.64247 , -130.85854 , -135.53024 , -150.

## Comparisson with other tools

### Biopython

In [7]:
import Bio.PDB
struct = Bio.PDB.PDBParser().get_structure("test", "../../../test.pdb")
model = struct[0]
chain1, chain2 = list(model.get_chains())

In [8]:
%timeit [Bio.PDB.Polypeptide.Polypeptide(chain).get_phi_psi_list() for chain in model]

44.8 ms ± 48.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit Bio.PDB.Polypeptide.Polypeptide(chain1).get_phi_psi_list()

21.5 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### MDTraj

In [10]:
import mdtraj as md

In [11]:
pdb = md.load_pdb("../../../test.pdb")
pdb

<mdtraj.Trajectory with 1 frames, 1867 atoms, 236 residues, without unitcells at 0x7f5e4d1f1eb8>

In [12]:
%timeit md.compute_phi(pdb)

666 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Pure python with MDTraj atom selection

In [13]:
%%timeit
backbone=pdb.top.select("backbone")
xyz=pdb.xyz[0]
result = np.empty(int(backbone.size/4))
for i in range(int(backbone.size/4)-1):
    result[i]=dihedral(
        xyz[backbone[i*4+2]], xyz[backbone[(i+1)*4]], xyz[backbone[(i+1)*4+1]], xyz[backbone[(i+1)*4+2]])

8.08 ms ± 59.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## It seems like ProStruct is still falling behind with the Python extensions! But the C++ library is much faster than other tools!